## 데이터 불러오기

In [ ]:
### 필요한 라이브러리 임폴트
import numpy as np
import pandas as pd

In [ ]:
### 구글 서버와 내 드라이브 연결하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### 현재 작업 경로 변경하기
%cd '/content/drive/MyDrive/KDT/비정형텍스트분석'

/content/drive/MyDrive/KDT/비정형텍스트분석


In [ ]:
### 저장된 데이터 불러오기

# 파일 경로 설정
train_okt_file_path = 'train_df_okt.csv'
train_mecab_file_path = 'train_df_mecab.csv'
test_okt_file_path = 'test_df_okt.csv'
test_mecab_file_path = 'test_df_mecab.csv'

# pd.read_csv() --> 데이터프레임 생성
train_df_okt = pd.read_csv(train_okt_file_path)
train_df_mecab = pd.read_csv(train_mecab_file_path)
test_df_okt = pd.read_csv(test_okt_file_path)
test_df_mecab = pd.read_csv(test_mecab_file_path)

# 결과 확인하기
print("Okt 형태소 분석기로 처리된 학습용 데이터 :")
print(train_df_okt)
print('-'*80)
print('Mecab 형태소 분석기로 처리된 학습용 데이터')
print(train_df_mecab)
print('-'*80)
print('OKt 형태소 분석기로 처리된 평가용 데이터')
print(test_df_okt)
print('-'*80)
print('Mecab 형태소 분석기로 처리된 평가용 데이터')
print(test_df_mecab)

Okt 형태소 분석기로 처리된 학습용 데이터 :
                                                 document  label
0                                          더빙 진짜 짜증나다 목소리      0
1                            포스터 보고 초딩 영화 오버 연기 조차 가볍다 않다      1
2                                       무재 밓었 다그 래서 보다 추천      0
3                             교도소 이야기 구먼 솔직하다 재미 없다 평점 조정      0
4       사이 몬페 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 ...      1
...                                                   ...    ...
147403                                           인간 문제 죄인      0
147404                                           평점 너무 낮다      1
147405                             한국인 거들다 먹거리 필리핀 혼혈 착하다      0
147406                              청춘 영화 최고봉 방황 우울하다 자화상      1
147407                           한국 영화 최초 수간 하다 내용 담기다 영화      0

[147408 rows x 2 columns]
--------------------------------------------------------------------------------
Mecab 형태소 분석기로 처리된 학습용 데이터
                                                 document 

## RandomForest 모델을 이용한 분류

### Word2Vec을 활용한 문장의 행렬화 

In [ ]:
### 필요한 라이브러리 임폴트
from gensim.models.word2vec import Word2Vec

In [ ]:
### 학습용 / 평가용 데이터 병합 --> concat()

# Okt 형태소 분석기로 처리한 데이터
df_okt = pd.concat([train_df_okt, test_df_okt], ignore_index=True)
print(f'Okt 형태소 분석기로 처리한 데이터를 병합한 결과 : \n{df_okt}')

print('-'*80)

# Mecab 형태소 분석기로 처리한 데이터
df_mecab = pd.concat([train_df_mecab, test_df_mecab], ignore_index=True)
print(f'Mecab 형태소 분석기로 처리한 데이터를 병합한 결과 : \n{df_mecab}')

Okt 형태소 분석기로 처리한 데이터를 병합한 결과 : 
                                                 document  label
0                                          더빙 진짜 짜증나다 목소리      0
1                            포스터 보고 초딩 영화 오버 연기 조차 가볍다 않다      1
2                                       무재 밓었 다그 래서 보다 추천      0
3                             교도소 이야기 구먼 솔직하다 재미 없다 평점 조정      0
4       사이 몬페 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 ...      1
...                                                   ...    ...
196508                  오랜 평점 기다 하다 킹왕짱 쌈뽕 영화 만나다 강렬하다 쾌함      1
196509             의지 박약 이나 하다 탈영 일단 주인공 김대희 닮다 이등병 찌다 따다      0
196510                      그림 좋다 완성 도도 높다 보다 내내 불안하다 만들다      0
196511             절대 보다 서다 되다 영화 재미 없다 기분 치고 하다 세트 에서 해먹      0
196512                                             마무리 이래      0

[196513 rows x 2 columns]
--------------------------------------------------------------------------------
Mecab 형태소 분석기로 처리한 데이터를 병합한 결과 : 
                                              

In [ ]:
### X, y 데이터 생성 --> document 컬럼과 label 컬럼 --> 넘파이 배열로 변환
X_data_okt = df_okt.document.values
y_data_okt = df_okt.label.values
X_data_mecab = df_mecab.document.values
y_data_mecab = df_mecab.label.values

In [ ]:
### document --> 리뷰 문장 --> 형태소 단위로 분할 --> 리스트 데이터 생성

# Okt 형태소 분석기로 처리한 데이터
sentences_okt = []

for review in X_data_okt:
    sentences_okt.append(review.split())

# Mecab 형태소 분석기로 처리한 데이터
sentences_mecab = []

for review in X_data_mecab:
    sentences_mecab.append(review.split())

In [ ]:
# 결과 확인하기
sentences_okt[:2]

[['더빙', '진짜', '짜증나다', '목소리'],
 ['포스터', '보고', '초딩', '영화', '오버', '연기', '조차', '가볍다', '않다']]

In [ ]:
# 결과 확인하기
sentences_mecab[:2]

[['진짜', '짜증', '네요', '목소리'],
 ['포스터', '보고', '초딩', '영화', '오버', '연기', '조차', '가볍', '구나']]

In [ ]:
### skip-gram 모델 학습 --> 임베딩 벡터 생성

# Okt 형태소 분석기로 처리한 데이터
model_sg_okt = Word2Vec(sentences=sentences_okt, vector_size=300, window=5, min_count=1, sg=1)

# Mecab 형태소 분석기로 처리한 데이터
model_sg_mecab = Word2Vec(sentences=sentences_mecab, vector_size=300, window=5, min_count=1, sg=1)

In [ ]:
### 학습된 모델 저장하기
model_sg_okt.save('model.sg_okt')
model_sg_mecab.save('model.sg_mecab')

In [ ]:
### 학습된 모델 불러오기
loaded_model_okt = Word2Vec.load('model.sg_okt')
loaded_model_mecab = Word2Vec.load('model.sg_mecab')

In [ ]:
### 생성된 단어 사전과 임베딩 벡터 확인하기

# 단어 사전 확인 --> model.wv.key_to_index
vocab_okt = loaded_model_okt.wv.key_to_index
print(f'Okt 형태소 분석기로 처리한 단어 사전 : \n{vocab_okt}')
print(f'Okt 형태소 분석기로 처리한 단어 사전의 단어 수  : \n{len(vocab_okt)}')
print('-'*80)
vocab_mecab = loaded_model_mecab.wv.key_to_index
print(f'Mecab 형태소 분석기로 처리한 단어 사전 : \n{vocab_mecab}')
print(f'Mecab 형태소 분석기로 처리한 단어 사전의 단어 수  : \n{len(vocab_mecab)}')

Okt 형태소 분석기로 처리한 단어 사전 : 
{'영화': 0, '보다': 1, '하다': 2, '없다': 3, '이다': 4, '있다': 5, '좋다': 6, '너무': 7, '정말': 8, '재밌다': 9, '되다': 10, '진짜': 11, '같다': 12, '으로': 13, '아니다': 14, '않다': 15, '에서': 16, '만들다': 17, '나오다': 18, '평점': 19, '연기': 20, '최고': 21, '생각': 22, '스토리': 23, '드라마': 24, '감동': 25, '사람': 26, '보고': 27, '이렇다': 28, '아깝다': 29, '배우': 30, '감독': 31, '재미있다': 32, '그냥': 33, '재미': 34, '시간': 35, '내용': 36, '까지': 37, '주다': 38, '재미없다': 39, '자다': 40, '하고': 41, '지루하다': 42, '쓰레기': 43, '가다': 44, '들다': 45, '모르다': 46, '그렇다': 47, '싶다': 48, '사랑': 49, '작품': 50, '알다': 51, '다시': 52, '하나': 53, '마지막': 54, '이건': 55, '정도': 56, '오다': 57, '완전': 58, '많다': 59, '처음': 60, '장면': 61, '액션': 62, '주인공': 63, '이렇게': 64, '안되다': 65, '차다': 66, '나다': 67, '최악': 68, '이야기': 69, '지금': 70, '별로': 71, '넘다': 72, '연출': 73, '느낌': 74, '좋아하다': 75, '인데': 76, '명작': 77, '역시': 78, '그리고': 79, '받다': 80, '많이': 81, '남다': 82, '이해': 83, '이런': 84, '괜찮다': 85, '이영화': 86, '라고': 87, '느끼다': 88, '버리다': 89, '때문': 90, '여자': 91, '이나': 92, '부터': 93, '아름답다': 94, '해

In [ ]:
### reveiw --> 문장 1개 --> 행렬화 함수 정의
def get_features(words, model, num_features):
    # 단어의 임베딩 벡터와 같은 크기 --> 0으로 채워진 벡터 생성
    feature_vector = np.zeros(num_features)
    # 입력되는 단어의 수를 세기 위한 초기값 설정
    num_words=0
    # 단어 사전 --> key(morpheme) 리스트 생성
    key_list = list(model.wv.key_to_index.keys())   

    for word in words:
        if word in key_list:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    # 문장 1개 --> 임베딩 벡터의 합 / 단어 수 --> 임베딩 벡터의 평균 값 추출
    feature_vector = np.divide(feature_vector, num_words)

    return feature_vector

In [ ]:
### review --> 전체 문장의 행렬화 함수 정의
def get_dataset_okt(reviews, model=loaded_model_okt, num_features=300):
    dataset = list()
    for review in reviews:
        ave_embeddings = get_features(review, model, num_features)
        dataset.append(ave_embeddings)

    reviewFeatureVecs = np.stack(dataset)

    return reviewFeatureVecs

In [ ]:
### review --> 전체 문장의 행렬화 함수 정의
def get_dataset_mecab(reviews, model=loaded_model_mecab, num_features=300):
    dataset = list()
    for review in reviews:
        ave_embeddings = get_features(review, model, num_features)
        dataset.append(ave_embeddings)

    reviewFeatureVecs = np.stack(dataset)

    return reviewFeatureVecs

In [ ]:
### review --> 문장 행렬 추출
reviews_vec_okt = get_dataset_okt(sentences_okt) 
reviews_vec_mecab = get_dataset_mecab(sentences_mecab)

# 결과 확인하기
print(f'Okt 형태소 분석기로 처리한 문장 행렬의 모양 : {reviews_vec_okt.shape}')
print('-'*80)
print(f'Mecab 형태소 분석기로 처리한 문장 행령의 모양 : {reviews_vec_mecab.shape}')

Okt 형태소 분석기로 처리한 문장 행렬의 모양 : (196513, 300)
--------------------------------------------------------------------------------
Mecab 형태소 분석기로 처리한 문장 행령의 모양 : (194701, 300)


In [ ]:
### 결과 저장하기
np.save('reviews_vec_okt.npy', reviews_vec_okt)
np.save('reviews_vec_mecab.npy', reviews_vec_mecab)

In [ ]:
### 결과 불러오기
reviews_vec_okt = np.load('reviews_vec_okt.npy')
reviews_vec_mecab = np.load('reviews_vec_mecab.npy')

In [ ]:
# print(reviews_vec_okt[0])
# print(reviews_vec_mecab[0])

### 학습용 / 평가용 데이터 생성

In [ ]:
### Okt 형태소 분석기로 처리한 데이터
X_train_okt = reviews_vec_okt[:147408]
X_test_okt = reviews_vec_okt[147408:]
y_train_okt = train_df_okt.label.values
y_test_okt = test_df_okt.label.values

# 결과 확인하기
print(f'Okt 학습용 데이터의 모양 : {X_train_okt.shape}')
print(f'Okt 학습용 레이블의 모양 : {y_train_okt.shape}')
print('-'*80)
print(f'Okt 평가용 데이터의 모양 : {X_test_okt.shape}')
print(f'Okt 평가용 레이블의 모양 : {y_test_okt.shape}')

Okt 학습용 데이터의 모양 : (147408, 300)
Okt 학습용 레이블의 모양 : (147408,)
--------------------------------------------------------------------------------
Okt 평가용 데이터의 모양 : (49105, 300)
Okt 평가용 레이블의 모양 : (49105,)


In [ ]:
### Mecab 형태소 분석기로 처리한 데이터
X_train_mecab = reviews_vec_mecab[:146005]
X_test_mecab = reviews_vec_mecab[146005:]
y_train_mecab = train_df_mecab.label.values
y_test_mecab = test_df_mecab.label.values

# 결과 확인하기
print(f'Mecab 학습용 데이터의 모양 : {X_train_mecab.shape}')
print(f'Mecab 학습용 레이블의 모양 : {y_train_mecab.shape}')
print('-'*80)
print(f'Mecab 평가용 데이터의 모양 : {X_test_mecab.shape}')
print(f'Mecab 평가용 레이블의 모양 : {y_test_mecab.shape}')

Mecab 학습용 데이터의 모양 : (146005, 300)
Mecab 학습용 레이블의 모양 : (146005,)
--------------------------------------------------------------------------------
Mecab 평가용 데이터의 모양 : (48696, 300)
Mecab 평가용 레이블의 모양 : (48696,)


### 모델 생성 / 학습 / 평가

In [ ]:
# 필요한 라이브러리 임폴트
from sklearn.ensemble import RandomForestClassifier

# 기본 모델 객체 생성
rf = RandomForestClassifier(random_state=99)

# n_estimators=300 모델 객체 생성
rf_300 = RandomForestClassifier(n_estimators=300, random_state=99)

In [ ]:
# Okt 형태소 분석기 데이터 --> 기본 모델 학습
rf.fit(X_train_okt, y_train_okt)

RandomForestClassifier(random_state=99)

In [ ]:
# Mecab 형태소 분석기 데이터 --> 기본 모델 학습
rf.fit(X_train_mecab, y_train_mecab)

RandomForestClassifier(random_state=99)

In [ ]:
# Okt 형태소 분석기 데이터 --> rf_300 모델 학습
rf_300.fit(X_train_okt, y_train_okt)

RandomForestClassifier(n_estimators=300, random_state=99)

In [ ]:
# Okt 형태소 분석기 데이터 --> 기본 모델 --> 평가용 데이터를 이용한 예측
pred_test_okt = rf.predict(X_test_okt)

In [ ]:
# Mecab 형태소 분석기 데이터 --> 기본 모델 --> 평가용 데이터를 이용한 예측
pred_test_mecab = rf.predict(X_test_mecab)

In [ ]:
# Okt 형태소 분석기 데이터 --> rf_300 모델 --> 평가용 데이터를 이용한 예측
pred_test_okt_300 = rf_300.predict(X_test_okt)

In [ ]:
### Okt 형태소 분석기 데이터 --> 기본 모델 --> 정확도 평가

# 필요한 라이브러리 임폴트
from sklearn.metrics import accuracy_score

# 평가용에 대한 정확도 평가
accuracy_test_okt = accuracy_score(y_test_okt, pred_test_okt)

# 결과 확인하기
print(f'정확도 = {accuracy_test_okt}')

정확도 = 0.8168618266978923


In [ ]:
### Mecab 형태소 분석기 데이터 --> 기본 모델 --> 정확도 평가

# 필요한 라이브러리 임폴트
from sklearn.metrics import accuracy_score

# 평가용에 대한 정확도 평가
accuracy_test_mecab = accuracy_score(y_test_mecab, pred_test_mecab)

# 결과 확인하기
print(f'정확도 = {accuracy_test_mecab}')

정확도 = 0.8056924593395761


In [ ]:
### Okt 형태소 분석기 데이터 --> rf_300 모델 --> 정확도 평가

# 필요한 라이브러리 임폴트
from sklearn.metrics import accuracy_score

# 평가용에 대한 정확도 평가
accuracy_test_okt_300 = accuracy_score(y_test_okt, pred_test_okt_300)

# 결과 확인하기
print(f'정확도 = {accuracy_test_okt_300}')

정확도 = 0.8191630180226046


## LightGBM 모델을 이용한 분류

### Word2Vec을 활용한 문장의 행렬화

### 모델 생성 / 학습 / 평가 

In [ ]:
# 필요한 라이브러리 임폴트
from lightgbm import LGBMClassifier

# 기본 모델 객체 생성
lgbm_okt = LGBMClassifier(random_state=99)
lgbm_mecab = LGBMClassifier(random_state=99)

# n_estimators=150, learning_rate=0.3 --> lgbm_150 모델 객체 생성
lgbm_150 = LGBMClassifier(n_stimators=150, learning_rate=0.3, random_state=99)

In [ ]:
# Okt 형태소 분석 데이터 --> 기본 모델 학습 
lgbm_okt.fit(X_train_okt, y_train_okt)

LGBMClassifier(random_state=99)

In [ ]:
# Okt 형태소 분석 데이터 --> 평가용 데이터를 이용한 예측
pred_test_okt = lgbm_okt.predict(X_test_okt)